In [2]:
import pandas as pd

In [3]:
d = pd.read_csv('auto-mpg.csv')
d.origin.replace({1: 'america', 2: 'europe', 3: 'japan'})
d.origin = pd.Categorical(d.origin)

In [4]:
X = d.drop('mpg', axis='columns')
y = d.mpg

In [5]:
categorical_columns = [
    col for col in X.columns if d[col].dtype == 'category'
]

numerical_columns = [   
    col for col in X.columns if col not in categorical_columns
]

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

In [7]:
def get_model(features):
    categorical_features = ['origin'] if 'origin' in features else []
    numerical_features = [col for col in features if col != 'origin']

    linear_preprocessing = ColumnTransformer(
        transformers = [
            ('categorial', OneHotEncoder(), categorical_features),
            ('numerical', StandardScaler(), numerical_features)
        ]
    )

    model = make_pipeline(
        linear_preprocessing,
        LinearRegression()
    )
    return model

In [8]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [9]:
from itertools import combinations
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
from numpy.typing import ArrayLike
from typing import List

In [10]:
class Solution:
    features: List[int]
    mse: float

    def __init__(self, y: ArrayLike):
        self.features = list()
        self.mse = mean_squared_error(y, [y.mean()]*len(y))

    def update(self, features: List[int], mse: float) -> bool:
        if(mse < self.mse):
            print(', '.join(str(x) for x in features))
            print(f"\tNew Error: {mse:.3f} better than {self.mse:.3f}")
            self.features = features
            self.mse = mse
            return True

        return False

## Feature Selection by complete enumeration

In [11]:
best = Solution(y)
features = X.columns
features

Index(['cylinders', 'displacement', 'hp', 'weight', 'acceleration', 'year',
       'origin'],
      dtype='object')

In [12]:
for n_features in range(1,len(features)+1):
    subsets = combinations(features, n_features)

    for subset in subsets:
        Xr = X[list(subset)]
        mses = cross_val_score(estimator=get_model(subset), X=Xr, y=y, cv=kfold, scoring='neg_mean_squared_error')
        mse = -np.average(mses)
        best.update(features=subset, mse=mse)

cylinders
	New Error: 24.131 better than 60.763
displacement
	New Error: 21.525 better than 24.131
weight
	New Error: 18.833 better than 21.525
cylinders, weight
	New Error: 18.605 better than 18.833
cylinders, year
	New Error: 17.465 better than 18.605
displacement, year
	New Error: 15.979 better than 17.465
weight, year
	New Error: 11.847 better than 15.979
weight, year, origin
	New Error: 11.365 better than 11.847
displacement, weight, year, origin
	New Error: 11.342 better than 11.365
displacement, hp, weight, year, origin
	New Error: 11.195 better than 11.342


In [13]:
print('Selected features: ', end='')
print(', '.join([str(x) for x in best.features]))

Selected features: displacement, hp, weight, year, origin


## Forward stepwise selection

In [14]:
all_features = features
current_features = list()
best = Solution(y)

In [15]:
while len(current_features) != len(all_features):
    selected_feature = None

    for feature in set(all_features) - set(current_features):
        new_features = current_features + [feature]
        Xr = X[new_features]
        mses = cross_val_score(estimator=get_model(new_features), X=Xr, y=y, cv=kfold, scoring='neg_mean_squared_error')
        mse = -np.average(mses)

        if(best.update(features=new_features, mse=mse)):
            selected_feature = feature
        
    if(selected_feature):
        current_features.append(selected_feature)
    else:
        break

hp
	New Error: 24.129 better than 60.763
weight
	New Error: 18.833 better than 24.129
weight, acceleration
	New Error: 18.470 better than 18.833
weight, year
	New Error: 11.847 better than 18.470
weight, year, origin
	New Error: 11.365 better than 11.847
weight, year, origin, displacement
	New Error: 11.342 better than 11.365
weight, year, origin, displacement, hp
	New Error: 11.198 better than 11.342


## Backward Stepwise selection

In [16]:
all_features = features
current_features = all_features
best = Solution(y)

In [17]:
while len(current_features)>0:
    selected_feature = None

    for feature in current_features:
        new_features = current_features.drop(feature)
        Xr = X[new_features]
        mses = cross_val_score(estimator=get_model(new_features), X=Xr, y=y, cv=kfold, scoring='neg_mean_squared_error')
        mse = -np.average(mses)

        if(best.update(features=new_features, mse=mse)):
            selected_feature = feature
        
    if(selected_feature):
        current_features = current_features.drop(selected_feature)
    else:
        break

displacement, hp, weight, acceleration, year, origin
	New Error: 11.396 better than 60.763
cylinders, displacement, hp, weight, year, origin
	New Error: 11.251 better than 11.396
displacement, hp, weight, year, origin
	New Error: 11.195 better than 11.251
